In [1]:
import os
import re

We will use in our implementation the function dict_merge_sum from the exercise 1 of our chapter on
dictionaries:

In [2]:
def dict_merge_sum(d1, d2):
    """Two dicionaries d1 and d2 with numerical value and 
    possiblly disjoint keys are merged and the values are added if 
    the exist in both values, otherwise the missing value is take n to be 0"""
    return { k: d1.get(k,0) + d2.get(k,0) for k in set(d1) | set(d2)}

d1 = dict(a=4, b=5, d=8)
d2 = dict(a=1, d=10, e=9)

dict_merge_sum(d1, d2)

{'a': 5, 'e': 9, 'd': 18, 'b': 5}

### Bag Of Word Class

In [3]:
class BagOfWords(object):
    """Implementing a bag of words, words corresponding with their
    frequency of usages in a document for usage by the Document calss, category class and the pool class"""
    
    def __init__(self):
        self.__number_of_words = 0
        self.__bag_of_words = {}
        
    def __add__(self, other):
        #Overloading of the + operator to joint two BagofWords
        erg = BagOfWords()
        erg.__bag_of_words = dict_merge_sum(self.__bag_of_words,
                                           other.__bag_of_words)
        return erg
    
    def add_word(self, word):
        #A word is added in the dictionary _bag_of_words
        self.__number_of_words +=1
        if word in self.__bag_of_words:
            self.__bag_of_words[word] +=1
        else:
            self.__bag_of_words[word] = 1
            
    def len(self):
        #returning the number of different words of and object
        return len(self.__bag_of_words)
    
    def Word(self):
        #Returning a list of the word containted in the object
        return self.__bag_of_words.keys()
    
    def BagOfWords(self):
        #Returning the dictionary, containing the word (keys)
        # with therir frequency (values)
        return self.__bag_of_words
    
    def WordFreq(self, word):
        #returning the frequency of a word
        if word in self.__bag_of_words:
            return self.__bag_of_words[word]
        else:
            return 0

### The Document Class

In [16]:
class Document(object):
    """Used both for learning (training) documents and for testing 
    documents. The optional parameter learing has to be set
    to True, if a classificator should be trained.
    If it is a test document learn has to be set to Flase."""
    
    _vocabulary = BagOfWords()
    
    def __init__(self, vocabulary):
        self.__name = ""
        self.__document_class = None
        self._words_and_freq = BagOfWords()
        Document._vocabulary = vocabulary
        
    def read_document(self,filename,learn=False):
        """A document is read. it is assumed that the document is either
        encoded in utf-8 or in iso-8859....(latin-1).
        The words of the document are stored in a Bag of Words,
        i.e. self._word_and_freq = BagOfWords()"""
        
        try:
            text = open(filename,"r", encoding="utf-8").read()
        except UnicodeDecodeError:
            text = open(filename,"r", encoding='latin-1').read()
            
        text = tect.lower()
        words = re.split(r"\w", text)
        
        self._number_of_words = 0
        for word in words:
            self._words_and_freq.add_word(word)
            if learn:
                Document._vocabulary.add_word(word)
                
    def __add__(self, other):
        # Overloading the "+" operator. adding two document consiss in 
        #adding the BagOfWords of the Documents
        res = Document(Document._vocabulary)
        res._words_and_freq = self._words_and_freq + other._words_and_freq
        
        return res
    
    def vocabulary_length(self):
        #returning the lenth of the vocaulary
        return len(Document._vocabulary)
    
    def WordsAndFreq(self):
        # Returning the dictionary, containing the words(keys)
        # with their ffrequency (values) as contrained in the 
        #bagOfWords attrubute of the documents
        
        return self._words_and_freq.BagOfWords()
    
    def Words(self):
        # Returning the words of hte document object
        d  = self._words_and_freq.BagOfWords()
        return d.keys()
    
    def WordFreq(self, word):
        #returning the number of times the word 'word' appeard in the document
        bow = self._words_and_freq.BagOfWords()
        if word in bow:
            return bow[word]
        else:
            return 0
        
    def __and__(self, other):
        ### Intersection of two documents, A list of words occuring 
        # in both documents is returned
        intersection = []
        words1 = self.Words()
        for word in other.Words():
            if word in words1:
                intersection += [word]
                
        return intersection
    

### Category / Collections of Documents

This is the class consisting of the documents for one category /class. We use the term category instead of
"class" so that it will not be confused with Python classes:

In [17]:
class Category(Document):
    def __init__(self, vocabulary):
        Document.__init__(self, vocabulary)
        self._number_of_docs = 0
        
    def Probability(self, word):
        #return the probability of the word 'word' given the class'self'
        
        voc_len = Document._vocabulary.len()
        SumN = 0
        for i in range(voc_len):
            SumN = Category._vocabulary.WordFreq(word)
        N = self._words_and_freq.WordFreq(word)
        erg = 1 + N
        erg  /= voc_len + SumN
        return erg
    
    def __add__(self, other):
        #Overloading the "+" operator. Adding two Category ohject
        #consists in adding the BagOfWords of the category objects
        res = Category(self._vocabulary)
        res._words_and_freq = self._words_and_freq + other._words_and_freq
        
        return res
    def SetNumberOfDocs(self, number):
        self._number_of_docs = number
        
    def NumberOfDocuments(self):
        return self._number_of_docs
    

### The Pool Class

The pool is the class, where the document classes are trained and kept

In [18]:
class Pool(object):
    def __init__(self):
        self.__document_classes = {}
        self.__vocabulary = BagOfWords()
        
    def sum_words_in_class(self, dclass):
        # The number of times all different word of a dclass appeaer in a class
        sum = 0
        for word in self.__vocabulary.Words():
            WaF = self.__document_classes[dclass].WordsAndFreq()
            if word in WaF:
                sum += WaF[word]
                
        return sum
    
    def learn(self, directory, dclass_name):
        # directory is a path, where the files of the class
        #with name dclass_name can be found
        x = Category(self.__vocabulary)
        dir = os.listdir(directory)
        for file in dir :
            d = Document(self.__vocabulary)
            #print(directory + "/" + file)
            d.read_document(directory + "/" + file, learn = True)
            x = x + d
            
        self.__document_classes[dclass_name] = x
        x.SetNumberOfDocs(len(dir))
        
    def Probability(self, doc, dclass= ""):
        #Calculates the probability for a class dclass given a document doc
        if dclass:
            sum_dclass = self.sum_words_in_class(dclass)
            prob = 0
            
            d = Document(self.__vocabulary)
            d.read_document(doc)
            
            for j in self.__document_classes:
                sum_j = self.sum_words_in_class(j)
                prob = 1
                for i in d.Words():
                    wf_dclass = 1 + self.__document_classes[dclass].WordsFreq(i)
                    wf = 1 + self.__document_classes[j].WordsFreq(i)
                    r = wf * sum_dclass / (wf_dclass * sum_j)
                    prod *=r
                prob += prod * self.__document_classes[j].NumberOfDocuments() / self.__document_classes[dclass].NumberOfDocuments()
                
            if prob != 0:
                return 1 / prob
            else:
                return -1
        else: 
            prob_list = []
            for dclass in self.__document_classes:
                prob = self.Probability(doc, dclass)
                prob_list.append([dclass, prob])
            
            prob_list.sort(key = lambda x:x[1], reverse= True)
            return prob_list
        
    def DocumentIntersectionWithClasses(self, doc_name):
        res = [doc_name]
        for dc in self.__document_classes:
            d = Document(self.__vocabulary)
            d.read_document(doc_name, learn=False)
            o = self.__document_classes[dc] & d
            intersection_ratio = len(o) / len(d.Words())
            res += (dc, intersection_ratio)
        return res

### Using The Classifier

In [ ]:
import os

DClasses = ["clinton",  "lawyer",  "math",  "medical",  "music",  "sex"]

base = "data/jokes/learn/"
p = Pool()
for dclass in DClasses:
    p.learn(base + dclass, dclass)



base = "data/jokes/test/"
results = []
for dclass in DClasses:
    dir = os.listdir(base + dclass)
    for file in dir:
        res = p.Probability(base + dclass + "/" + file)
        results.append(f"{dclass}: {file}: {str(res)}")
        
print(results[:10])